In [15]:
import pandas as pd

# 指定Excel文件路径
excel_file_path = '00_raw_data\\1_test_zb_df_2 t.xlsx'

# 读取Excel文件,并自动设置行索引和列索引-
#df = pd.read_excel(excel_file_path, index_col=0)

# 查看数据
#print(df)

In [16]:
#提取偶数列，属性的权重列，并将偶数列（权重列）删除，返回删除权重后的dataframe
def getWeight(df):
    #获取偶数列列名
    even_columns = df.columns[1::2]
    #提取偶数列的数据
    even_columns_data = df[even_columns]    
    #删除偶数列的数据
    newdf = df.drop(columns=even_columns)
    
    #将偶数列的列索引改成和奇数列索引一样
    even_columns_data.set_axis(newdf.columns, axis=1, inplace=True)

    # python可以有多个返回值，如果，只对一个感兴趣，
    # 可以用_来接受，表示不感兴趣
    return even_columns_data, newdf

#data_weight, data = getWeight(df)
#print(data)
#print(data_weight)


In [17]:
#对数据进行正则化处理
def data_Normalization(df, weight_data):
    #选出值全为0和值全为100的列
    columns_to_drop = df.columns[(df == 0).all() | (df == 100).all()]
    # 创建新的DataFrame，包含有效的列
    new_df = df.drop(columns=columns_to_drop)
    # 删除权重对应的列
    weight = weight_data.drop(columns = columns_to_drop)

    # 对整个dataframe进行归一化，归一化的方式是所有数据除以100
    normalized_df = new_df / 100
    return normalized_df, weight


#normalization_df, weight = data_Normalization(data, data_weight)

In [21]:

file_path = '01_normalization_data\\1_test_zb_df_2t.xlsx'
def saveExcel(weight, normalization_df, file_path):
    #对权重进行去重
    weight = weight.drop_duplicates()
    result_data = pd.concat([weight, normalization_df], ignore_index=True)
    print(result_data)
    #将权重加入到数据的第一行
    result = result_data.rename(index={0: 'weight'})
    #print(result)
    #保存
    result.to_excel(file_path, index=True)
    return result

#saveExcel(weight, normalization_df, file_path)

In [22]:
#定义列表，对三个文件进行批量处理
raw_file = [
     '00_raw_data/1_test_zb_df_2 t.xlsx',
     '00_raw_data/2_test_zb_df_2 t.xlsx',
     '00_raw_data/3_test_zb_df_2 t.xlsx',
     '00_raw_data/人员数据.xlsx'
]

result_file = [
    '01_normalization_data/01_test_zb_df_2 t.xlsx',
    '01_normalization_data/02_test_zb_df_2 t.xlsx',
    '01_normalization_data/03_test_zb_df_2 t.xlsx',
    '01_normalization_data/人员数据.xlsx'
]

sum_data = []
#定义数据处理的函数
def dataMung(raw_path, result_path):
    # 读取Excel文件,并自动设置行索引和列索引-
    df = pd.read_excel(raw_path, index_col=0)
    data_weight, data = getWeight(df)
    normalization_df, weight = data_Normalization(data, data_weight)
    result = saveExcel(weight, normalization_df, result_path)
    sum_data.append(result)#将每个文件处理得到的结果保存在sum_data数组中，方便后面聚类使用

for i in range(4):
    dataMung(raw_file[i], result_file[i])
#dataMung('00_raw_data/人员数据.xlsx', '01_normalization_data/people.xlsx')
    

       XNZJF01000001  XNZJF01000006  XNZJF01060063  XNZJF02010004  \
0              1.361            2.4            0.6          0.215   
1              0.100            0.5            0.6          0.800   
2              0.100            0.5            0.6          0.800   
3              0.100            0.5            0.6          0.800   
4              0.100            0.5            0.6          0.800   
...              ...            ...            ...            ...   
9997           0.100            0.5            0.6          0.800   
9998           0.100            0.5            0.6          0.800   
9999           0.100            0.5            0.6          0.800   
10000          0.100            0.5            0.6          0.800   
10001          0.100            0.5            0.6          0.800   

       XNZJF02010006  XNZJF02010009  XNZJF02020001  XNZJF02020002  \
0              2.144          1.074            1.6          1.588   
1              0.500          0.4

In [20]:
# 选择方差排在前20的属性组成新的数据 
def choice_top20_var(data, result_path):
    # 获取数据的weight行
    weight = data.iloc[0]
    # 获取除了weight行外的其他数据
    attributes_data = data.iloc[1:]
    # 计算属性的方差，并按照方差进行排序
    variances = attributes_data.var()
    # 按照方差进行排序
    sorted_variances = variances.sort_values(ascending=False)
    # 提取前20个属性的索引
    top_20_attributes = sorted_variances[:20].index
    # 根据提取到的索引，从原始数据中选出包含前20个属性的信息，并提取对应的weight权重
    selected_data = attributes_data[top_20_attributes]
    print(selected_data)
    selected_weight = weight[top_20_attributes]
    print(selected_weight)

    # 将前20的属性和和对应的权重weight组合成新的dataframe
    final_data = pd.concat([selected_weight, selected_data])

    # 将新的dataFrame保存为CSV格式
    final_data.to_excel(result_path, index=True)

    
choice_top20_var(sum_data[1], result_file[1])

       ZJF02010002  ZJF02010001  ZJF02060051  ZJF02070004  ZJF02070002  \
1              0.0          0.0     1.000000          0.0          0.7   
2              0.0          0.0     1.000000          0.4          0.7   
3              1.0          0.0     1.000000          0.0          0.7   
4              0.0          0.0     1.000000          0.0          0.7   
5              1.0          1.0     1.000000          0.4          1.0   
...            ...          ...          ...          ...          ...   
10097          0.0          0.0     0.700000          0.0          0.7   
10098          0.0          1.0     0.568750          0.4          0.7   
10099          0.0          0.0     0.700000          0.0          0.7   
10100          0.0          0.0     1.000000          0.4          0.7   
10101          1.0          1.0     0.004861          0.7          0.5   

       ZJF02060038  ZJF02060001  ZJF02060040  ZJF02060034  ZJF02060062  \
1              0.7         0.60     0

In [8]:
#根据方差对数据进行聚类
import numpy as np
from sklearn.cluster import KMeans

#挑选出标准差最大的20个数据，作为聚类的簇中心
def select_data(data):
    #参数为一个dataframe

    data_1 = data
    #data_1 = sum_data[0]
    #print(data_1)
    # 1. 去掉DataFrame中的第一行数据
    data_1 = data_1.drop(index='weight')
    #print(data_1)
    # 2. 对DataFrame的列数据集按照每一列的标准差进行排序
    sorted_columns = data_1.std().sort_values(ascending=False).index

    # 3. 选择标准差最大的20列数据作为聚类的中心
    selected_columns = sorted_columns[:20]
    selected_data = data_1[selected_columns]
    #print("select_data:", selected_data)

    # 4. 将数据转化为numpy数组方便后面聚类
    #此时initial_centers的shape为(10001, 20)
    #要想让20个数据做为簇中心， 得转换为(20, 10001)
    initial_centers = selected_data.to_numpy()
    transposed_initial_center = initial_centers.transpose()
    X = data_1.to_numpy()
    transposed_X = X.transpose()

    return transposed_initial_center, transposed_X

def myKmeans(data):
    '''
        参数：data 一个dataframe
    '''
    # 1. 计算出这个dataframe的方差最大的20个列，作为簇中心
    transposed_initial_centers, transposed_X = select_data(sum_data[0])
    # print("initial_center", initial_centers)
    # print("X", X)

    # 2. 创建KMeans对象，设置聚类中心的个数
    n_clusters = 20  # 假设聚类中心的个数为5
    kmeans = KMeans(n_clusters=n_clusters, init=transposed_initial_centers)

    # 执行聚类
    kmeans.fit(transposed_X)

    # 获取聚类中心
    centroids = kmeans.cluster_centers_

    # 获取每个样本所属的簇
    labels = kmeans.labels_

    #print("聚类中心：")
    #print(centroids)
    print("样本的簇标签：")
    print(labels)
    labels = np.array(labels)
    count = np.bincount(labels, minlength=20)
    for i, count in enumerate(count):
        print(f"数字{i}出现了{count}次")

for i in range(3):
    myKmeans(sum_data[i])


c:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1307: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params(X)
c:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1307: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params(X)
c:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting

样本的簇标签：
[ 7  4  8 16  4  4  8  8  8 16 16 16 19 16  8  8 16  8 16 16 16 16 16 16
 16 16 16 16 16 16 16  8 16 16  8 16  8 16  8  8 16 16 16  7 16 19 19 19
  8 16 16 16 16 16 16 16 16 16  8 16 16 16 16  8 16 16  7  8 16  8  1  0
 18 19  4  4 14 12 14 15 15 16 19 16  4  8 16  8 16 16 16 16 16 16 16 16
 16 11 16  2 16  8 16 16  9 16  6 17 13 13 16 16  3 19  5 16 19 19 19 18
 19 16 16 16 16 16 13 16 15 19 10 16 16 16  7 16  7 19  4 16 16 18 18 18
 16 16 16 16 16 16 16 18  4 18 19  7  7 16 19 19]
数字0出现了1次
数字1出现了1次
数字2出现了1次
数字3出现了1次
数字4出现了8次
数字5出现了1次
数字6出现了1次
数字7出现了7次
数字8出现了20次
数字9出现了1次
数字10出现了1次
数字11出现了1次
数字12出现了1次
数字13出现了3次
数字14出现了2次
数字15出现了3次
数字16出现了83次
数字17出现了1次
数字18出现了7次
数字19出现了16次
样本的簇标签：
[ 7  4  8 16  4  4  8  8  8 16 16 16 19 16  8  8 16  8 16 16 16 16 16 16
 16 16 16 16 16 16 16  8 16 16  8 16  8 16  8  8 16 16 16  7 16 19 19 19
  8 16 16 16 16 16 16 16 16 16  8 16 16 16 16  8 16 16  7  8 16  8  1  0
 18 19  4  4 14 12 14 15 15 16 19 16  4  8 16  8 16 16 16 16 16 16 16 16
 16 11 16  